### Step 1: get the data

Need to do the following:
* find the data source I'll use (Here vs Google Maps vs Mapbox)
* generate random points across both cities for trip start and end points
* generate the transit and driving trips and store them in a geojson file locally. Each trip needs to contain the start and end points, the path, the start time and end time.

In [1]:
import os
HERE_API_KEY = "TDwv81Ze1NKUD2GFnjJa0jGi0nSwdUUY2WANH6kAMKk"#os.environ["HERE_API_KEY"]

In [2]:
import requests

In [5]:
response = requests.get(f"https://transit.router.hereapi.com/v8/routes?apiKey={HERE_API_KEY}&origin=41.79457,12.25473&destination=41.90096,12.50243")

In [8]:
response.content

b'{"routes":[{"id":"R007993-C0","sections":[{"id":"R007993-C0-S0","type":"pedestrian","departure":{"time":"2020-10-30T18:32:00+01:00","place":{"type":"place","location":{"lat":41.79457,"lng":12.25473}}},"arrival":{"time":"2020-10-30T18:38:00+01:00","place":{"name":"Fiumicino Aeroporto","type":"station","location":{"lat":41.793466,"lng":12.251997},"id":"415713377"}},"transport":{"mode":"pedestrian"}},{"id":"R007993-C0-S1","type":"transit","departure":{"time":"2020-10-30T18:38:00+01:00","place":{"name":"Fiumicino Aeroporto","type":"station","location":{"lat":41.793466,"lng":12.251997},"id":"415713377"}},"arrival":{"time":"2020-10-30T19:10:00+01:00","place":{"name":"Roma Termini","type":"station","location":{"lat":41.90054,"lng":12.502168},"id":"415712984"}},"transport":{"mode":"regionalTrain","name":"LE","category":"Regional Train","color":"#4C8DC1","textColor":"#000000","headsign":"Roma Termini"},"agency":{"id":"aDmtre00","name":"Trenitalia","website":"http://www.trenitalia.com/"},"attr

In [3]:
import osmnx as ox
import numpy as np
import datetime

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [100]:
gdf_seattle_buildings[['geometry']]

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,geometry
0,POINT (-122.35474 47.53407)
1,POINT (-122.33444 47.51528)
2,POINT (-122.29053 47.53910)
3,POINT (-122.32048 47.62568)
4,POINT (-122.32431 47.62177)
...,...
205724,"POLYGON ((-122.30490 47.65287, -122.30495 47.6..."
205725,"POLYGON ((-122.40032 47.65936, -122.40032 47.6..."
205726,"MULTIPOLYGON (((-122.30108 47.71176, -122.3011..."
205727,"MULTIPOLYGON (((-122.38730 47.57759, -122.3868..."


In [101]:
gdf_seattle_buildings[['geometry']].to_file("data/seattle_buildings.geojson", driver="GeoJSON")

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
import mapbox

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/mapbox/utils.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Sequence


In [106]:
mapbox.Datasets.create({"name": "seattle-buildings"})

AttributeError: 'dict' object has no attribute 'baseuri'

In [4]:
gdf_seattle_buildings = ox.geometries_from_place('Seattle, Washington, USA', {'building': True})

In [ ]:
gdf_vancouver_buildings = ox.geometries_from_place('Vancouver, British Columbia, Canada', {'building': True})

In [96]:
# https://developer.here.com/documentation/public-transit/dev_guide/index.html

In [111]:
datetime.datetime.utcnow().isoformat()

def format_time(year, month, day, hour, minute):
    return datetime.datetime(year, month, day, hour, minute).isoformat() + "-07:00"
format_time(2020, 10, 30, 9, 15)

'2020-10-30T09:15:00-07:00'

In [40]:
import geopy
import json

def transit_trip(start_geom, end_geom, departure_time):
    origin_lat_lon = f"{start_geom.coords[0][1]},{start_geom.coords[0][0]}"
    destination_lat_lon = f"{end_geom.coords[0][1]},{end_geom.coords[0][0]}"
    url = f"https://transit.router.hereapi.com/v8/routes?departureTime={departure_time}&apiKey={HERE_API_KEY}&origin={origin_lat_lon}&destination={destination_lat_lon}&return=polyline"
    response = requests.get(url)
    trip = {'transit_response': json.loads(response.content), 'transit_url': url}
    return trip

def car_trip(start_geom, end_geom, departure_time):
    origin_lat_lon = f"{start_geom.coords[0][1]},{start_geom.coords[0][0]}"
    destination_lat_lon = f"{end_geom.coords[0][1]},{end_geom.coords[0][0]}"
    url = f"https://router.hereapi.com/v8/routes?transportMode=car&departureTime={departure_time}&apiKey={HERE_API_KEY}&origin={origin_lat_lon}&destination={destination_lat_lon}&return=polyline"
    response = requests.get(url)
    trip = {'car_response': json.loads(response.content), 'car_url': url}
    return trip


def random_start_end(gdf):
    N_points = len(gdf)
    start_idx, end_idx = np.random.randint(0, N_points, 2)
    start_geom = gdf.iloc[start_idx].geometry
    end_geom = gdf.iloc[end_idx].geometry
    # reject anything less than 1500m https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3377942/
    if geopy.distance.geodesic(start_geom.coords[0][::-1], end_geom.coords[0][::-1]).meters < 1500:
        return random_start_end(gdf)
    return start_geom, end_geom
    
    
def run_experiment(start_geom, end_geom, departure_time):
    parameters = {
        'start': start_geom, 
        'end': end_geom, 
        'departure_time': departure_time
    }
    results = {
        **car_trip(start_geom, end_geom, departure_time),
        **transit_trip(start_geom, end_geom, departure_time)
    }
    return {**parameters, **results}

In [57]:
# 7.30 and 9, 5 and 6.30
# do 7.30, 7.35, 8, 8.05, 8.30, 8.35, 9, 9.05

times = [
    f'2020-10-28T{h:02d}:{m:02d}:00-07:00' for (h, m) in [(8, 0), (8, 5), (8, 30), (8, 35), (17, 0), (17, 5), (17, 30), (17,35), (18, 0), (18, 5)]
]
len(times)

10

In [58]:
times

['2020-10-28T08:00:00-07:00',
 '2020-10-28T08:05:00-07:00',
 '2020-10-28T08:30:00-07:00',
 '2020-10-28T08:35:00-07:00',
 '2020-10-28T17:00:00-07:00',
 '2020-10-28T17:05:00-07:00',
 '2020-10-28T17:30:00-07:00',
 '2020-10-28T17:35:00-07:00',
 '2020-10-28T18:00:00-07:00',
 '2020-10-28T18:05:00-07:00']

In [95]:
gdf_nodes = gdf_seattle_buildings.loc[gdf_seattle_buildings.element_type == "way"]

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
import geopy

geopy.distance.geodesic()

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Distance(0.0)

In [20]:
location_pairs = [random_start_end(gdf_nodes) for i in range(10)]

In [97]:
gdf_nodes

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,unique_id,osmid,element_type,source,geometry,amenity,description,name,railway,wheelchair,...,name:etymology,bus,bench,use,bin,name:es,alt_ref,ways,source:building:levels:underground,museum
38,way/4712871,4712871,way,data.seattle.gov,"POLYGON ((-122.31526 47.59305, -122.31526 47.5...",NaN,NaN,Pacific Tower,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,way/4712941,4712941,way,NaN,"POLYGON ((-122.29470 47.57546, -122.29470 47.5...",fire_station,NaN,Fire Station 30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,way/4725281,4725281,way,NaN,"POLYGON ((-122.33023 47.59839, -122.33016 47.5...",NaN,NaN,King Street Station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,way/12903132,12903132,way,Yahoo aerial imagery;local knowledge,"POLYGON ((-122.34931 47.62034, -122.34926 47.6...",NaN,NaN,Space Needle,NaN,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,way/13866261,13866261,way,King County GIS;data.seattle.gov,"POLYGON ((-122.33024 47.61203, -122.33024 47.6...",NaN,NaN,Washington State Convention Center,NaN,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205420,way/864532966,864532966,way,NaN,"POLYGON ((-122.37658 47.68165, -122.37647 47.6...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205421,way/865154900,865154900,way,NaN,"POLYGON ((-122.38549 47.67188, -122.38549 47.6...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205422,way/865154901,865154901,way,NaN,"POLYGON ((-122.38549 47.67177, -122.38549 47.6...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205423,way/865154902,865154902,way,NaN,"POLYGON ((-122.38105 47.67337, -122.38105 47.6...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
experiments = []
errors = []
for start_geom, end_geom in location_pairs:
    for departure_time in times:
        try:
            experiments.append(run_experiment(start_geom, end_geom, departure_time))
        except Exception as exception:
            errors.append(exception)


In [62]:
import pickle 

In [63]:
with open("data/progress_capture_raw.pckl", "wb") as fp:
    pickle.dump(experiments, fp)

In [64]:
with open("data/progress_capture_raw.pckl", "rb") as fp:
    experiments = pickle.load(fp)

In [65]:
experiment = experiments[0]


In [69]:
departure_time = datetime.datetime.fromisoformat(experiment['departure_time'])
transit_arrival_time = datetime.datetime.fromisoformat(experiment["transit_response"]["routes"][0]['sections'][-1]["arrival"]["time"])
car_arrival_time = datetime.datetime.fromisoformat(experiment["car_response"]["routes"][0]['sections'][-1]["arrival"]["time"])
diff = transit_arrival_time - car_arrival_time 
ratio_transit_to_car = (transit_arrival_time - departure_time)/(car_arrival_time - departure_time)

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
experiment['car_response']['routes'][0]['sections'][0]['polyline']

'BGmur46Cn7vppHuhBMTsvCoGAoBAoLUsiBUsJT8GT0K3DwCnBsJ_E0P3IgKrEgK7BwMA0oBA4DA03BAsOAgKAkITkInBoLjDsTzF0tBvMoanGkS_EwM_E4IjD4I3D0FjD8BnBgKzF4NjI0KvHgF3DsEjD4IvH4I_JwHnL8G7L4IrO0F_JwCrEkD_EsJzP4D7GkI7LgF7GoB7BoGvHwHnG4D7BgFvC4DnBsEnB4DA4NAsiBU4cU4SAwHA4NU0jBUoVAofUsdUoGA8LU8GAsTAsJA0UUoQU8G8BoG8BgZsJsTrT4IrJ8L7LoL7LU_iBAnVAvWAnfArOAjhBAvbUrYUjSAzFAjcUrdA3mBA3XArYA_TA_JA7QA_OA7VA3IArJAnVAnGTzeAnLUrOA7LAnkBA3DA72CAzPTvqBA3SAjXAvWAjXA3mBU3wBA7GU3mBU_OA7aAztBU7kBU3IAvHAzKA7BAnGTnL0PrOsOjNsErE0K_J8kBriBwHnGkX7V0UrT4X7VgUjS8QzPgUjS0FzFoa3X8QzPgKrJs2BrxB8VnVwH3I0FnG4DnG8B_E8B7GkD7L8B7G8BzF8B_EkD7GkDzF4S7f4SvgBoGnLkSrdwHvM8G7L4D7GkD7G4D_J4DzKkD7G4DvHoGnLsE7GoQvb8BjDwC_EouBjuCgPzZwMnVoQvbwCrEkD_EwCrE4c3wBoQjcoLzUwRvgBkDnGwC_E0KvWoG7LoG7LkI_OsYzoBgPzZsOrYgK_OgFjIoB7BkDnG8BjDsOrYoBvC0FvH4DjD8B7BgFnB0FwCwCoBwC8BwCwCgF0F0FwH8B8BwCkDkIgKkNwR4IoL8BwC4N8QsE0FgFjI8pBjpCokBz8BkNvWsdrxBgK3SsJjS4Sze8L3SgFoGsE8BkhBUA4DAwgBAouBjgBR'

In [81]:
import polyline

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [89]:
polyline = experiment['car_response']['routes'][0]['sections'][0]['polyline']
coords = list(iter_decode(polyline))
coords

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(47.584995, -122.314676),
 (47.58553, -122.31467),
 (47.58552, -122.3134),
 (47.58562, -122.3134),
 (47.58564, -122.3134),
 (47.58582, -122.31339),
 (47.58637, -122.31338),
 (47.58652, -122.31339),
 (47.58663, -122.3134),
 (47.5868, -122.31346),
 (47.58684, -122.31348),
 (47.58699, -122.31356),
 (47.58724, -122.3137),
 (47.5874, -122.31377),
 (47.58756, -122.3138),
 (47.58776, -122.3138),
 (47.58841, -122.3138),
 (47.58847, -122.3138),
 (47.58936, -122.3138),
 (47.58959, -122.3138),
 (47.58975, -122.3138),
 (47.58988, -122.31381),
 (47.59001, -122.31383),
 (47.59019, -122.31388),
 (47.5905, -122.31397),
 (47.59123, -122.31417),
 (47.59165, -122.31427),
 (47.59194, -122.31435),
 (47.59214, -122.31443),
 (47.59228, -122.31448),
 (47.59242, -122.31454),
 (47.59251, -122.31459),
 (47.59254, -122.31461),
 (47.5927, -122.3147),
 (47.59292, -122.31483),
 (47.59309, -122.31495),
 (47.59317, -122.31501),
 (47.59324, -122.31506),
 (47.59338, -122.31518),
 (47.59352, -122.31534),
 (47.59364, -12

In [87]:
from here_polyline.flexpolyline.decoding import iter_decode

/Users/patrick/.virtualenvs/jupyter/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
